- <mark> 弹性网络模型效果差 </mark>
- <mark> 遍历所有可能？ </mark>
- <mark> 是否需要进行预剪枝，使用Boosting等操作 </mark>
- 集成算法和回归加权
## import包

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

## 数据准备

In [3]:
# 读取数据
df_all = pd.read_csv('/Users/wangjingran/Desktop/PTEN mutation/L on ML/paras.txt', sep='\t')
# 数据切分
df_ASD_Cancer = df_all[df_all['Disease'].isin(['ASD', 'Cancer'])]
df_ASD_AC = df_all[df_all['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_Cancer_AC = df_all[df_all['Disease'].isin(['Cancer', 'ASD_Cancer'])]
def average(lst):
    if len(lst) == 0:
        return 0
    else:
        return sum(lst) / len(lst)

## 模型循环选择——ASD versus Cancer

In [ ]:

# 生成全部的组合
import itertools
variables = ['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient']
all_combinations = []
# 组合
for r in range(1, len(variables)+1):
    combinations = list(itertools.combinations(variables, r))
    combinations = [list(combination) for combination in combinations]  # 转换为列表
    all_combinations.extend(combinations)

for combination in all_combinations:
    # 筛选数据
    X_select = df_AA_ASD_Cancer[combination]
    y_select = df_AA_ASD_Cancer['Disease']
    # 切分后的数据集重新将index排序
    X_select = X_select.reset_index(drop=True)
    y_select = y_select.reset_index(drop=True)
    shuffle_index = np.random.permutation(X_select.index)
    X_select= X_select.iloc[shuffle_index]
    y_select = y_select.iloc[shuffle_index]
    # 切分数据集
    X_train_select = X_select[:650]
    X_test_select = X_select[650:]
    y_train_select = y_select[:650]
    y_test_select = y_select[650:]
    # 训练模型
    rf_clf_select = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=2000, learning_rate=0.5, random_state=42)
    rf_clf_select.fit(X_train_select, y_train_select)
    score_select = rf_clf_select.feature_importances_
    y_pred_select = rf_clf_select.predict(X_test_select)
    # 打印输出
    print("-"*20,end = '')
    print(str(combination),end = '')
    print("-"*20)
    print("*****重要性得分*****")
    print(score_select)
    print("*****模型准确性*****")
    print(accuracy_score(y_test_select, y_pred_select))



## 模型循环选择——ASD versus ASD_Cancer

In [4]:
# 生成全部的组合
import itertools
variables = ['Co.evolution','Conservation','Entropy','RASA','energy',
             'Betweenness','Closeness','Degree','Eigenvector',
             'Clustering.coefficient','Effectiveness','Sensitivity',
             'MSF','DFI','Stiffness']
all_combinations = []
# 组合
for r in range(1, len(variables)+1):
    combinations = list(itertools.combinations(variables, r))
    combinations = [list(combination) for combination in combinations]  # 转换为列表
    all_combinations.extend(combinations)

list_ASD_AC = []
df_ASD_Cancer = df_all[df_all['Disease'].isin(['ASD', 'Cancer'])]
df_ASD_AC = df_all[df_all['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_Cancer_AC = df_all[df_all['Disease'].isin(['Cancer', 'ASD_Cancer'])]

  

In [7]:
for combination in tqdm(all_combinations):
    # 选取数据 
    X_select = df_ASD_AC[combination]
    y_select = df_ASD_AC['Disease']
    # 切分后的数据集重新将index排序
    roc_select_list = []
    for i in range(10):
        X_select = X_select.reset_index(drop=True)
        y_select = y_select.reset_index(drop=True)
        shuffle_index = np.random.permutation(X_select.index)
        X_select= X_select.iloc[shuffle_index]
        y_select = y_select.iloc[shuffle_index]
        # 切分数据集
        X_train_select = X_select[:round(X_select.shape[0] * 0.7)]
        X_test_select = X_select[round(X_select.shape[0] * 0.7):]
        y_train_select = y_select[:round(X_select.shape[0] * 0.7)]
        y_test_select = y_select[round(X_select.shape[0] * 0.7):]
        # 训练模型————随机森林
        rf_clf_select = RandomForestClassifier(n_estimators=1000, n_jobs=-1, max_depth=5)
        rf_clf_select.fit(X_train_select, y_train_select)
        score_select = rf_clf_select.feature_importances_
        #y_pred_select = rf_clf_select.predict(X_test_select)
        y_pred_select_prob = rf_clf_select.predict_proba(X_test_select)[:,1]
    
        # 计算ROC值
        label_encoder = LabelEncoder()
        y_test_select_bi = label_encoder.fit_transform(y_test_select)
        # y_pred_select_bi = label_encoder.fit_transform(y_pred_select)
        fpr_select, tpr_select, thresholds_select = roc_curve(y_test_select_bi, y_pred_select_prob)
        roc_auc_select = auc(fpr_select, tpr_select)
        roc_select_list.append(roc_auc_select)

    f = open("随机森林_ASD_ASD_Cancer.txt", "a")
    f.write(str(combination) + '\t')
    f.write(str(average(roc_select_list)) + '\t')
    f.write(str(roc_select_list))
    f.write('\n')
    f.close()

  1%|▏         | 436/32767 [42:03<51:59:20,  5.79s/it] 


KeyboardInterrupt: 

## 模型循环选择——Cancer versus ASD_Cancer

In [8]:
for combination in tqdm(all_combinations):
    # 选取数据 
    X_select = df_Cancer_AC[combination]
    y_select = df_Cancer_AC['Disease']
    # 切分后的数据集重新将index排序
    roc_select_list = []
    for i in range(10):
        X_select = X_select.reset_index(drop=True)
        y_select = y_select.reset_index(drop=True)
        shuffle_index = np.random.permutation(X_select.index)
        X_select= X_select.iloc[shuffle_index]
        y_select = y_select.iloc[shuffle_index]
        # 切分数据集
        X_train_select = X_select[:round(X_select.shape[0] * 0.7)]
        X_test_select = X_select[round(X_select.shape[0] * 0.7):]
        y_train_select = y_select[:round(X_select.shape[0] * 0.7)]
        y_test_select = y_select[round(X_select.shape[0] * 0.7):]
        # 训练模型————梯度下降
        rf_clf_select = GradientBoostingClassifier(n_estimators=1000, max_depth=5)
        rf_clf_select.fit(X_train_select, y_train_select)
        score_select = rf_clf_select.feature_importances_
        #y_pred_select = rf_clf_select.predict(X_test_select)
        y_pred_select_prob = rf_clf_select.predict_proba(X_test_select)[:,1]
        # 计算ROC值
        label_encoder = LabelEncoder()
        y_test_select_bi = label_encoder.fit_transform(y_test_select)
        # y_pred_select_bi = label_encoder.fit_transform(y_pred_select)
        fpr_select, tpr_select, thresholds_select = roc_curve(y_test_select_bi, y_pred_select_prob)
        roc_auc_select = auc(fpr_select, tpr_select)
        roc_select_list.append(roc_auc_select)

    f = open("模型选择/梯度下降_Cancer_ASD_Cancer.txt", "a")
    f.write(str(combination) + '\t')
    f.write(str(average(roc_select_list)) + '\t')
    f.write(str(roc_select_list))
    f.write('\n')
    f.close()

 10%|▉         | 3221/32767 [29:56:09<274:36:06, 33.46s/it]  


KeyboardInterrupt: 